In [1]:
#Import the dependent libraries
from citipy import citipy
from datetime import datetime
import pandas as pd
import numpy as np
import timeit
import time
import requests

# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
#Starting URL for weather Map API calls
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [4]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(5)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1 
    
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | puerto ayora
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | carnarvon
Processing Record 4 of Set 1 | samarai
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | saint-joseph
Processing Record 7 of Set 1 | aklavik
Processing Record 8 of Set 1 | toliary
City not found. Skipping...
Processing Record 9 of Set 1 | avarua
Processing Record 10 of Set 1 | saskylakh
Processing Record 11 of Set 1 | egvekinot
Processing Record 12 of Set 1 | san cristobal
Processing Record 13 of Set 1 | chistogorskiy
Processing Record 14 of Set 1 | pangnirtung
Processing Record 15 of Set 1 | severo-kurilsk
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | kapaa
Processing Record 18 of Set 1 | georgetown
Processing Record 19 of Set 1 | shimoda
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | mataura
Processing Record 22 of Set 1 | burica
Cit

Processing Record 30 of Set 4 | upata
Processing Record 31 of Set 4 | kavaratti
Processing Record 32 of Set 4 | asau
Processing Record 33 of Set 4 | frenkendorf
Processing Record 34 of Set 4 | chernyy yar
Processing Record 35 of Set 4 | banmo
City not found. Skipping...
Processing Record 36 of Set 4 | thompson
Processing Record 37 of Set 4 | sierpc
Processing Record 38 of Set 4 | chokurdakh
Processing Record 39 of Set 4 | bodden town
Processing Record 40 of Set 4 | talara
Processing Record 41 of Set 4 | cayenne
Processing Record 42 of Set 4 | surt
Processing Record 43 of Set 4 | mae hong son
Processing Record 44 of Set 4 | hunchun
Processing Record 45 of Set 4 | jawhar
Processing Record 46 of Set 4 | buzmeyin
Processing Record 47 of Set 4 | buta
Processing Record 48 of Set 4 | seymchan
Processing Record 49 of Set 4 | leningradskiy
Processing Record 50 of Set 4 | karla
Processing Record 1 of Set 5 | iberia
Processing Record 2 of Set 5 | envira
Processing Record 3 of Set 5 | leh
Processi

Processing Record 26 of Set 8 | thessalon
Processing Record 27 of Set 8 | salamiyah
Processing Record 28 of Set 8 | mogadishu
Processing Record 29 of Set 8 | anloga
Processing Record 30 of Set 8 | burnie
Processing Record 31 of Set 8 | grindavik
Processing Record 32 of Set 8 | yarada
Processing Record 33 of Set 8 | irece
Processing Record 34 of Set 8 | armacao dos buzios
City not found. Skipping...
Processing Record 35 of Set 8 | iqaluit
Processing Record 36 of Set 8 | ossora
Processing Record 37 of Set 8 | boguchany
Processing Record 38 of Set 8 | birao
Processing Record 39 of Set 8 | yeppoon
Processing Record 40 of Set 8 | one hundred mile house
City not found. Skipping...
Processing Record 41 of Set 8 | novouzensk
Processing Record 42 of Set 8 | mangan
Processing Record 43 of Set 8 | yingkou
Processing Record 44 of Set 8 | irbit
Processing Record 45 of Set 8 | gumdag
Processing Record 46 of Set 8 | khandbari
Processing Record 47 of Set 8 | fort nelson
Processing Record 48 of Set 8 |

Processing Record 18 of Set 12 | tynda
Processing Record 19 of Set 12 | sao gabriel da cachoeira
Processing Record 20 of Set 12 | jati
Processing Record 21 of Set 12 | iquique
Processing Record 22 of Set 12 | deshnok
Processing Record 23 of Set 12 | la ronge
Processing Record 24 of Set 12 | haines junction
Processing Record 25 of Set 12 | tasbuget
City not found. Skipping...
Processing Record 26 of Set 12 | dingle
Processing Record 27 of Set 12 | maltahohe
Processing Record 28 of Set 12 | takoradi
Processing Record 29 of Set 12 | pangoa
Processing Record 30 of Set 12 | oktyabrskiy
Processing Record 31 of Set 12 | devils lake
Processing Record 32 of Set 12 | san andres
Processing Record 33 of Set 12 | constitucion
Processing Record 34 of Set 12 | luba
Processing Record 35 of Set 12 | svetlogorsk
Processing Record 36 of Set 12 | lagoa
Processing Record 37 of Set 12 | dajal
Processing Record 38 of Set 12 | khonuu
City not found. Skipping...
Processing Record 39 of Set 12 | warri
Processin

Processing Record 6 of Set 16 | santa eulalia del rio
City not found. Skipping...
Processing Record 7 of Set 16 | ozu
Processing Record 8 of Set 16 | padang
Processing Record 9 of Set 16 | surgut
Processing Record 10 of Set 16 | palabuhanratu
City not found. Skipping...
Processing Record 11 of Set 16 | casma
Processing Record 12 of Set 16 | umarizal
Processing Record 13 of Set 16 | vuktyl
-----------------------------
Data Retrieval Complete      
-----------------------------


In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,82.36,75,65,7.72,broken clouds
1,Busselton,AU,-33.6500,115.3333,58.08,73,7,10.58,clear sky
2,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,12.66,clear sky
3,Samarai,PG,-10.6167,150.6667,81.45,74,96,10.74,overcast clouds
4,Rikitea,PF,-23.1203,-134.9692,79.00,68,0,15.26,clear sky
5,Saint-Joseph,RE,-21.3667,55.6167,71.98,81,81,5.93,broken clouds
6,Aklavik,CA,68.2191,-135.0107,-19.37,69,75,10.36,broken clouds
7,Avarua,CK,-21.2078,-159.7750,82.45,78,20,14.97,few clouds
8,Saskylakh,RU,71.9167,114.0833,-9.11,97,52,7.56,broken clouds
9,Egvekinot,RU,66.3167,-179.1667,-24.02,86,8,9.33,clear sky


In [6]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")